# User for Genre

- User for genre toma como parámetro un género particular, y retorna el usuario  con mayor tiempo jugado para dicho genero.

- Para evitar consultas lentas, crearemos una tabla auxiliar que sera agrupada por año y jugador, donde agruparemos los generos en un str y sumaremos el tiempo jugado del usuario.

- **Observación:** No tenemos la información sobre el año en que un usuario jugó un juego, por lo que vamos a tomar el año de lanzamiento del juego como el año en que el usuario jugó el juego. 

In [21]:
import pandas as pd
import os

path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Carga de data

In [22]:
path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Cargamos la data necesaria para solucionar este problema.
steam_colums = ['id','genres','release_year']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)

user_item_cols = ['item_id','user_id','playtime_forever']
user_items = pd.read_csv(path_user, usecols = user_item_cols)


In [23]:
steam_games.head(3)

,genres,steam_id,release_year
0,"Strategy, Action, Indie, Casual, Simulation",761140.0,2018
1,"Free to Play, Strategy, Indie, RPG, Card Game,...",643980.0,2018
2,"Free to Play, Simulation, Sports, Casual, Indi...",670290.0,2017


In [24]:
user_items.head(3)

,user_id,playtime_forever,item_id
0,76561197970982479,0.0,10.0
1,76561197970982479,0.0,20.0
2,76561197970982479,0.0,30.0


## Creación de tabla consulta

- En esta sección combinaremos steam_games  con user_items, por medio de sus ids. Luego agruparemos por año de lanzamiento y grupo de genero comun, para calcular la suma de tiempos jugados por dicho genero.

In [25]:

steam_users = steam_games.merge(user_items,
                              left_on='steam_id', 
                              right_on= 'item_id',
                              how='inner').drop(columns=['steam_id','item_id'])

In [26]:
steam_users.sample(5)

,genres,release_year,user_id,playtime_forever
480255,"Open World, Action, Multiplayer, Crime, Third ...",2005,optomisticprime007,0.0
2016671,"Board Game, Simulation, Multiplayer, Sandbox, ...",2015,76561198062914222,0.0
3020404,"City Builder, Indie, Sandbox, Simulation, Stra...",2012,76561198059389948,0.0
912738,"Fighting, Gore, Action, Multiplayer, Violent, ...",2013,wolfclawe,0.0
1898769,"Action, Indie, Casual, Adventure, Puzzle",2015,phrostb,0.0


- Por eficiencia en la consulta vamos a agrupar por año de lanzamiento y usuario.
Notamos que quedan dos columans sin agrupar, genres (str) y playtime_forever. Al realizar la agrupación por suma, vamos a acumular en playtime_forever las horas jugadas por un usuario en un año determinado; 

- tambien vamos a quedarnos con los usuarios que jugaron al menos una hora durante todos los años.

- Por otra parte genres concatenará cada string que contiene las columnas agrupadas. Esta técnica nos permitira seguir teniendo toda la información necesaria. Posteriormente vamos a transformar la columna genres para eliminar los generos repetidos en cada registro.

In [27]:
tabla_endpoint_2 = steam_users.groupby(['release_year','user_id']).sum(numeric_only=False).reset_index()

- Así observamos que la tabla resultante tendra un tamaño considerablemente menor y cada usuario que se encuentre en estas lista, tendra los generos jugados por este usuario.

- Ahora vamos a eliminar los generos repetidos en cada registro, esto con el fin de reducir el tamaño del string de cada registro de genres. dado que esta consulta consume bastante memoria, utilicé una tecnica de carga con generadores.

In [28]:
## tomar los generos diferentes de cada uno de los caracteres, esto con el fin de reducir el tamaño del archivo.

def genres_unique(x):
  return ', '.join(set(map(str.strip, x.split(', '))))

tabla_endpoint_2['genres'] = list(genres_unique(fila) for fila in tabla_endpoint_2['genres'])

In [29]:
tabla_endpoint_2.head(10)

,release_year,user_id,genres,playtime_forever
0,1983,2Ta4,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
1,1983,76561197960324641,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
2,1983,76561197966936422,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
3,1983,76561197968887720,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
4,1983,76561197969020980,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
5,1983,76561197971401137,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
6,1983,76561197971951483,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
7,1983,76561197972452208,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
8,1983,76561197973470219,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0
9,1983,76561197975369524,"Short, Cartoon, Difficult, Action, Arcade, Ret...",0.0


In [30]:
# Me quedo con los usuarios que el tiempo de juego sea positivo
tabla_endpoint_2 = tabla_endpoint_2[tabla_endpoint_2['playtime_forever']>0]

## Exportar tabla consulta 2

In [31]:
path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data_v2.csv.gz')

tabla_endpoint_2.to_csv(path_endpoint_2, index = False,compression='gzip',)

In [32]:
tabla_endpoint_2

,release_year,user_id,genres,playtime_forever
122,1987,76561197966936422,"Pixel Graphics, Retro, Point & Click, 1990's, ...",0.6
123,1987,76561197967174031,"Pixel Graphics, Retro, Point & Click, 1990's, ...",1.2
1190,1988,comquack,"Turn-Based, Sci-fi, Singleplayer, RPG, Action,...",2.4
1579,1989,76561198014848587,"Retro, Strategy, Simulation, Adventure, Classic",1.2
1962,1990,76561198000121873,"Action, Sci-fi, Platformer, Singleplayer, 2D, ...",0.6
...,...,...,...,...
763388,2017,zwanzigdrei,"Difficult, Singleplayer, Exploration, Simulati...",0.6
763389,2017,zxcvbnm6995,"Singleplayer, Hack and Slash, Free to PlayFree...",3.0
763393,2017,zychicken,"Singleplayer, Platformer, Third-Person Shooter...",0.6
763399,2017,zzbrunozz,"Singleplayer, Third-Person Shooter, Simulation...",2.4


In [33]:
tabla_endpoint_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320238 entries, 122 to 763402
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   release_year      320238 non-null  int64  
 1   user_id           320238 non-null  object 
 2   genres            320238 non-null  object 
 3   playtime_forever  320238 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 12.2+ MB


## Lectura de tabla consulta

**Retornaremos el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año**

In [34]:
path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data_v2.csv.gz')
table_2 = pd.read_csv(path_endpoint_2,compression='gzip')

In [35]:
print(table_2.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320238 entries, 0 to 320237
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   release_year      320238 non-null  int64  
 1   user_id           320238 non-null  object 
 2   genres            320238 non-null  object 
 3   playtime_forever  320238 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 282.6 MB
None


## Construcción de la consulta
- A continuación vamos a realizar las consultas necesarias para encontrar el usuario con mas horas jugadas por genero.
Para responder este problema vamos a ejemplificar la consulta inspecciónar con un genero particular, en este caso **Action**, luego vamos a construir las consultas para este genero particular

In [36]:
genre = 'Action'

In [37]:
# Filtro la tabla para quedarme únicamente con registros que contengan el género deseado
genre_filter = table_2[table_2['genres'].str.contains(genre)]


In [38]:
genre_filter.head(4)

,release_year,user_id,genres,playtime_forever
2,1988,comquack,"Turn-Based, Sci-fi, Singleplayer, RPG, Action,...",2.4
4,1990,76561198000121873,"Action, Sci-fi, Platformer, Singleplayer, 2D, ...",0.6
5,1990,76561198041356854,"Action, Sci-fi, Platformer, Singleplayer, 2D, ...",0.6
9,1993,76561197966836671,"Difficult, Singleplayer, Turn-Based Strategy, ...",0.6


In [53]:
## Sumamos playtime_forever de cada usuario
df_sum_play_time = genre_filter.groupby('user_id')['playtime_forever'].sum()

In [54]:
pd.DataFrame(df_sum_play_time)

,playtime_forever
user_id,
--000--,2347.71
--ace--,1172.63
--ionex--,643.07
-2SV-vuLB-Kg,803.68
-404PageNotFound-,3311.12
...,...
zzonci,7.15
zzoptimuszz,1308.28
zzydrax,77.33


In [55]:
# Usuario con más horas jugadas en total
user_max_time = df_sum_play_time.idxmax()

print(f'El usurio con mas horas jugadas \npara el genero {genre} es {user_max_time}')


El usurio con mas horas jugadas 
para el genero Action es REBAS_AS_F-T


In [56]:
# Filtrar solo por los registros del usuario con mayor tiempo de juego y me quedo con las columnas release_year y playtime_forever
mask = genre_filter['user_id'] == user_max_time

user_filter = genre_filter[mask][['release_year', 'playtime_forever']]

In [57]:
user_filter.head(5)

,release_year,playtime_forever
17153,1996,0.00
36546,1998,0.02
50057,1999,3.75
64835,2000,0.00
79718,2001,0.18


In [58]:
# Este es el tiempo acumulado jugado por el usuario con mas tiempo jugado con el genero action.
pd.DataFrame(user_filter)

,release_year,playtime_forever
17153,1996,0.00
36546,1998,0.02
50057,1999,3.75
64835,2000,0.00
79718,2001,0.18
93224,2002,0.02
111602,2003,31.05
142419,2004,151.70
168447,2005,54.89
211548,2006,30.78


In [59]:
# Retornaremos como una lista de diccionarios del estilo
# {year : playtime_forever_cum }

target = user_filter.rename(columns={'release_year': 'year', 'playtime_forever': 'total_horas'}).to_dict(orient='records')

In [60]:
target[:5]

[{'year': 1996, 'total_horas': 0.0},
 {'year': 1998, 'total_horas': 0.02},
 {'year': 1999, 'total_horas': 3.75},
 {'year': 2000, 'total_horas': 0.0},
 {'year': 2001, 'total_horas': 0.18}]

In [61]:
print(f"Usuario con más horas jugadas para Género {genre}: es {user_max_time}, \nHoras jugadas:")
target

Usuario con más horas jugadas para Género Action: es REBAS_AS_F-T, 
Horas jugadas:


[{'year': 1996, 'total_horas': 0.0},
 {'year': 1998, 'total_horas': 0.02},
 {'year': 1999, 'total_horas': 3.75},
 {'year': 2000, 'total_horas': 0.0},
 {'year': 2001, 'total_horas': 0.18},
 {'year': 2002, 'total_horas': 0.02},
 {'year': 2003, 'total_horas': 31.05},
 {'year': 2004, 'total_horas': 151.7},
 {'year': 2005, 'total_horas': 54.89000000000001},
 {'year': 2006, 'total_horas': 30.78},
 {'year': 2007, 'total_horas': 173.75},
 {'year': 2008, 'total_horas': 215.04},
 {'year': 2009, 'total_horas': 1028.71},
 {'year': 2010, 'total_horas': 1215.96},
 {'year': 2011, 'total_horas': 3103.64},
 {'year': 2012, 'total_horas': 4459.36},
 {'year': 2013, 'total_horas': 6805.02},
 {'year': 2014, 'total_horas': 7992.97},
 {'year': 2015, 'total_horas': 17897.55},
 {'year': 2016, 'total_horas': 22049.68},
 {'year': 2017, 'total_horas': 584.46}]

## Función UserForGenre

In [7]:
import os
import pandas as pd
def user_for_genre(genre :str):
  path_endpoint_2 = os.path.join('..','data','clear','02_user_for_genre_data.csv.gz')
  table_2 = pd.read_csv(path_endpoint_2)

  ## Filtro la tabla para quedarme unicamente con registros que contengan el genero deseado
  genre_filter = table_2[table_2['genres'].str.contains(genre)]
  
  ## usuario con mayor tiempo de juego acumulado
  df_sum_play_time =  genre_filter[['user_id','playtime_forever']].groupby(['user_id']).sum()
  
  # usuario con mas horas jugadas en total
  if len(df_sum_play_time) > 0:
    user_max_time = df_sum_play_time.idxmax().iloc[0]
  else:
    return ('No existe usuario que haya jugado este juego')
  
  ## Filtraremos solo por los registros del usuario con mayor tiempo de juego
  ## nos quedamos con el año de lanzamiento y el tiempo de juego
  mask = genre_filter['user_id'] == user_max_time
  user_filter = genre_filter[mask][['release_year','playtime_forever']]
  
  # Sumamos las horas jugadas del usuario por añoprofile
  user_filter_sum = user_filter.groupby('release_year').sum().reset_index()
  
  target = user_filter_sum.rename(columns={'release_year':'year'})[['year', 'playtime_forever']].to_dict(orient='records')
  return {f"Usuario con más horas jugadas para Género {genre}" : user_max_time, "Horas jugadas": target}

In [9]:
user_for_genre('Difficult')


{'Usuario con más horas jugadas para Género Difficult': 'REBAS_AS_F-T',
 'Horas jugadas': [{'year': 2004, 'playtime_forever': 2.4},
  {'year': 2005, 'playtime_forever': 1.2},
  {'year': 2009, 'playtime_forever': 15.6},
  {'year': 2010, 'playtime_forever': 19.8},
  {'year': 2011, 'playtime_forever': 49.8},
  {'year': 2012, 'playtime_forever': 69.6},
  {'year': 2013, 'playtime_forever': 109.8},
  {'year': 2014, 'playtime_forever': 127.2},
  {'year': 2015, 'playtime_forever': 296.4},
  {'year': 2016, 'playtime_forever': 366.6},
  {'year': 2017, 'playtime_forever': 10.2}]}

In [10]:
user_items['playtime_forever'][user_items['user_id']=='REBAS_AS_F-T'].sum()

NameError: name 'user_items' is not defined